In [234]:
import pandas as pd
import numpy as np

data_with_infertility_path = "./data/PCOS_infertility.csv"
data_without_infertility_path = "./data/PCOS_data_without_infertility.xlsx"
df_infertility = pd.read_csv(data_with_infertility_path)
df_fertility = pd.read_excel(data_without_infertility_path, sheet_name="Full_new")

#### We have been provided two datasets. If the 2nd dataset has more information, we need to include that as well. Compare the values of each columns for 2 available datasets i.e., with infertility and without infertility
#### If they are equal then just consider the data with fertility because the this data contains the data from infertility as well as extra data


In [235]:
pcos = df_fertility["PCOS (Y/N)"].equals(df_infertility["PCOS (Y/N)"])
I_beta_hcg = df_fertility["  I   beta-HCG(mIU/mL)"].equals(df_infertility["  I   beta-HCG(mIU/mL)"])
II_beta_hcg = df_fertility["II    beta-HCG(mIU/mL)"] .equals(df_infertility["II    beta-HCG(mIU/mL)"])

# Converting AMH column of fertility data to match the datatype for equals comparison
df_fertility["AMH(ng/mL)"] = df_fertility["AMH(ng/mL)"].astype("str", copy=False)
amh = df_fertility["AMH(ng/mL)"].equals(df_infertility["AMH(ng/mL)"])

comparison_features = {"pcos":pcos, 
                        "I_beta_hcg": I_beta_hcg, 
                        "II_beta_hcg": II_beta_hcg, 
                        "amh": amh
                        }

conflicts_in = [k for k, v in comparison_features.items() if not v]
if len(conflicts_in):
    print(f"The columns where values do not match are: {conflicts_in}")
else:
    df = df_fertility
    print("All the data in all given columns are same")



The columns where values do not match are: ['II_beta_hcg']


#### Checking which rows of the mentioned columns have different data and what are they

In [236]:
def find_differences(fertility_col, infertility_col):
    info = list()
    # info.append(["row", "fertility data value", "infertility data value"])
    for i, v in enumerate(infertility_col):
        if fertility_col[i] != v:
            info.append([i, fertility_col[i], v])
    return info, fertility_col.name


II_beta_hcg_info, II_beta_hcg_col_name = find_differences(df_fertility["II    beta-HCG(mIU/mL)"], df_infertility["II    beta-HCG(mIU/mL)"])
for i in II_beta_hcg_info:
    print(f'For column {II_beta_hcg_col_name}, row {i[0]}, the fertility dataset value {i[1]} differs from the infertility dataset value {i[2]}')


For column II    beta-HCG(mIU/mL), row 123, the fertility dataset value 1.99. differs from the infertility dataset value 0.110416667


#### Replacing the value by averaging the column values of "II beta-HCG(mIU/mL)" where pcos is 1

In [237]:
df_fertility.at[123, "II    beta-HCG(mIU/mL)"] = np.NaN
req_vals_for_hcg_ii = [df_fertility.at[x, "II    beta-HCG(mIU/mL)"] for x in range(len(df_fertility["II    beta-HCG(mIU/mL)"])) if df_fertility.at[x, "PCOS (Y/N)"] == df_fertility.at[123, "PCOS (Y/N)"]]
df_fertility.at[123, "II    beta-HCG(mIU/mL)"] = np.nanmean(req_vals_for_hcg_ii)

#### Now we can set our default data frame as df_fertility

In [238]:
df = df_fertility
# Drop column "Unnamed: 44"
df = df.drop(columns=["Unnamed: 44"])
pd.set_option('display.max_columns', None)

#### Checking for null, nan and blank values

In [241]:
null_vals = np.where(pd.isna(df))
na_vals = df[df.isna().any(axis=1)]

cols_with_nan = [df.columns[x] for x in list(null_vals[1])]

print(null_vals)
print(list(null_vals[0])[0])
print(list(null_vals[1])[0])
# na_vals


(array([], dtype=int64), array([], dtype=int64))


IndexError: list index out of range

#### Now that we know that 'Fast food (Y/N)' and 'Marraige Status (Yrs)' column has the nan values, we work on eliminating or replacing it.

In [240]:
# df["Fast food (Y/N)"] = df["Fast food (Y/N)"].fillna(df["Fast food (Y/N)"].mean())
# df["Marraige Status (Yrs)"] = df["Marraige Status (Yrs)"].fillna(df["Marraige Status (Yrs)"].mean())
df["Fast food (Y/N)"] = df["Fast food (Y/N)"].interpolate(method='linear', axis=0).ffill().bfill()
df["Marraige Status (Yrs)"] = df["Marraige Status (Yrs)"].interpolate(method='linear', axis=0).ffill().bfill()
na_vals1 = df[df.isna().any(axis=1)]
col1_idx = df.columns.get_loc("Marraige Status (Yrs)")
col2_idx = df.columns.get_loc("Fast food (Y/N)")
print(col1_idx, col2_idx)
# See the new values for Nan
print(df.iloc[458, 13])
print(df.iloc[156, 35])

13 35
5.5
0.5
